In [1]:
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py
!pip install ltn "tensorflow==2.15.*" -q
!pip install tensorflow[and-cuda]==2.15.* --q
!pip install keras_tuner -q

--2025-05-07 05:26:33--  https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/logictensornetworks/logictensornetworks/master/examples/multiclass_classification/commons.py [following]
--2025-05-07 05:26:33--  https://raw.githubusercontent.com/logictensornetworks/logictensornetworks/master/examples/multiclass_classification/commons.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2147 (2.1K) [text/plain]
Saving to: ‘commons.py.2’

commons.py.2        100%[===================>]   2.1

In [2]:
!mkdir -p data
!gdown 1SGiYOyuSiirueZR3F6K0d7aHcPT43PYw -O ./data/dpp4-26-03-25-feat.parquet
!ls -l data/dpp4-26-03-25-feat.parquet
!mkdir -p results
!mkdir -p weights

Downloading...
From: https://drive.google.com/uc?id=1SGiYOyuSiirueZR3F6K0d7aHcPT43PYw
To: /content/data/dpp4-26-03-25-feat.parquet
100% 77.8M/77.8M [00:02<00:00, 28.2MB/s]
-rw-r--r-- 1 root root 77788177 Mar 31 04:26 data/dpp4-26-03-25-feat.parquet


In [3]:
import os
import ltn
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    matthews_corrcoef,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
    auc,
    ConfusionMatrixDisplay,
)

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from dataclasses import dataclass
tqdm.pandas()
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow import keras

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
ROOT_DIR = '/content'

In [6]:
# Data & Model training arguments
SEED = 42
LR = 0.00001
BATCH_SIZE = 128
FEAT_NAME = [
    'CDKextended',
    'ECFP',
    'ECFP_512',
    'ECFP_2048',
    # 'ChemBERT2_Embedding',
    # 'Llama3-2_Embedding',
    # 'MolWt',
    # 'LogP',
    # 'TPSA',
    # 'HBD',
    # 'HBA',
    # 'RotatableBonds',
]
MODEL_NAME = 'LTN'

TRAIN_HISTORY_PATH = os.path.join(ROOT_DIR, f'results/{"-".join(FEAT_NAME)}-{MODEL_NAME}.csv')
os.makedirs(os.path.dirname(TRAIN_HISTORY_PATH), exist_ok=True)

In [7]:
import pandas as pd
import os

# Define the correct path
file_path = os.path.join(ROOT_DIR, "data/dpp4-26-03-25-feat.parquet")

# Read the Parquet file
df_final = pd.read_parquet(file_path)

In [8]:
df_final.head()

,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,...,ECFP_512,Llama3-2_Embedding,ChemBERT2_Embedding,MolWt,LogP,TPSA,HBD,HBA,RotatableBonds,Physiochemical
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,...,"[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.49902532, 2.9190297, -0.23703773, -0.25883...","[-0.38957188, -0.6216704, 0.4703551, 0.1626513...",238.375,2.2966,46.33,1.0,2.0,3.0,"[238.37499999999994, 2.2966000000000006, 46.33..."
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.9239326, 2.7885501, 0.38945127, 0.3505689,...","[-0.3062373, -0.24003226, 0.14207578, 0.525902...",142.202,-0.0440,46.33,1.0,2.0,1.0,"[142.20199999999997, -0.04400000000000004, 46...."
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.41079238, 2.760039, 0.025078857, -0.101544...","[-0.3257358, -0.012994111, 0.17750713, -0.0522...",168.240,0.3608,32.34,1.0,2.0,1.0,"[168.23999999999998, 0.3608, 32.34, 1.0, 2.0, ..."
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.13337657, 2.8452492, 0.630767, 0.08860778,...","[-0.0054997415, -0.37361592, 0.015422794, 0.30...",184.308,1.1616,15.27,1.0,2.0,1.0,"[184.30799999999996, 1.1616, 15.27, 1.0, 2.0, ..."
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.6948919, 2.6412396, 0.3524785, -0.21108356...","[-0.6008385, -0.16555355, 0.1292091, 0.9035775...",185.227,-1.1885,89.42,2.0,3.0,3.0,"[185.22699999999995, -1.1885000000000001, 89.4..."


In [9]:
df_final.shape

(6563, 25)

In [10]:
# Operators and axioms

In [11]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [12]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes,units_values= [16,16,8], activation ='elu'):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(uv, activation=activation) for uv in units_values]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.25)

    def call(self, inputs, training=False):
        x = inputs
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)

        return self.dense_class(x)

def build_model(hp):

    units_value1 = hp.Choice("units_value1",[
        768,384,192,
        ])
    units_value2 = hp.Choice("units_value2",[
        384,192,
        ])
    units_value3 = hp.Choice("units_value3",[
        192,96,
        ])

    activations = hp.Choice("activation", ["relu", 'elu'])
    model = MLP(
        n_classes=2,
        units_values=[units_value1,units_value2,units_value3],
        activation=activations,
    )
    predicate = LTNModel(model)
    predicate.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate",[1e-5, 1e-4, 1e-3])
        ),
    )
    return predicate

In [13]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=4))

In [14]:
# Creating Object for Training
class LTNModel(tf.keras.Model):
    def __init__(self,input_model):
        super(LTNModel, self).__init__()

        self.model = input_model # accuracy prediction
        self.p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(self.model,single_label=True)) # satification, main ltn
        # self.p = ltn.Predicate.FromLogits(self.model, activation_function="softmax", with_class_indexing=True)
        self.train_sat = tf.keras.metrics.Mean(name='train_axiom_sat')
        self.test_sat = tf.keras.metrics.Mean(name='test_axiom_sat')
        self.train_accu = tf.keras.metrics.Accuracy(name='train_accuracy')
        self.test_accu = tf.keras.metrics.Accuracy(name='test_accuracy')

    @tf.function
    def axioms(self,features , labels,p,training = False):

        class_A = ltn.Constant(0, trainable=False)
        class_B = ltn.Constant(1, trainable=False)

        x_A = ltn.Variable("x_A",features[labels==0])
        x_B = ltn.Variable("x_B",features[labels==1])

        axioms = [
            Forall(x_A,p([x_A,class_A],training = training)),
            Forall(x_B,p([x_B,class_B],training = training)),
            ]
        for i in range(len(axioms)):
            # find nan float values and replace them with 0
            if tf.reduce_any(tf.math.is_nan(axioms[i].tensor)):
                axioms[i].tensor = tf.constant(0.0)

        sat_level = formula_aggregator(axioms).tensor
        return sat_level


    @tf.function
    def train_step(self,data_train):
        # sat and update
        x,y = data_train
        with tf.GradientTape(persistent = False) as tape:
            self.sat = self.axioms(x, y,self.p,training = True)
            self.loss_value = 1. -self.sat # Learning

        grads = tape.gradient(self.loss_value, self.p.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.p.trainable_variables))
        pred = tf.argmax(tf.nn.softmax(self.model(x)),axis =1)
        return {"Training Satisfaction Level":self.train_sat(self.axioms(x, y,self.p)),"Accuracy":self.train_accu(y,pred)}

    @tf.function
    def test_step(self,data ):
        x, y = data
        pred = tf.argmax(tf.nn.softmax(self.model(x)),axis =1)
        return {"Satisfaction Level" :self.test_sat(self.axioms(x, y,self.p)),"Accuracy":self.test_accu(y,pred)}

In [15]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import gc

BATCH_SIZE = 128
SEED = 42
EPOCHS = 100

X_train = np.concatenate([np.vstack(df_final[df_final['splits']=='train'][col]) for col in FEAT_NAME],axis = 1)
y_train = df_final[df_final['splits']=='train']['target']

X_test = np.concatenate([np.vstack(df_final[df_final['splits']=='test'][col]) for col in FEAT_NAME],axis = 1)
y_test = df_final[df_final['splits']=='test']['target']

X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

y_train = np.asarray(y_train).flatten()
y_test = np.asarray(y_test).flatten()

In [16]:
print(X_train.shape,y_train.shape)

(5250, 4608) (5250,)


In [17]:
ds_train = tf.data.Dataset.from_tensor_slices((X_train,y_train)).batch(BATCH_SIZE).shuffle(512,seed= SEED)
ds_test = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(BATCH_SIZE)

In [18]:
!rm -rf tuner_dir

In [19]:
# GridSearch, RandomSearch
tuner = kt.GridSearch(
    build_model,
    objective="val_Accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="tuner_dir",
    project_name="mlp",
)

# Step 4: Search best hyperparameters
tuner.search(ds_train, epochs=EPOCHS, validation_data=ds_test, verbose=1)

Trial 10 Complete [00h 01m 42s]
val_Accuracy: 0.9565879702568054

Best val_Accuracy So Far: 0.9725818634033203
Total elapsed time: 00h 16m 38s


In [20]:
tuner.get_best_hyperparameters()[0].values

{'units_value1': 768,
 'units_value2': 384,
 'units_value3': 192,
 'activation': 'relu',
 'learning_rate': 0.0001}

In [21]:
tuner.results_summary()

Results summary
Results in tuner_dir/mlp
Showing 10 best trials
Objective(name="val_Accuracy", direction="max")

Trial 01 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 192
activation: relu
learning_rate: 0.0001
Score: 0.9725818634033203

Trial 07 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 96
activation: relu
learning_rate: 0.0001
Score: 0.9725818634033203

Trial 08 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 96
activation: relu
learning_rate: 0.001
Score: 0.9695354104042053

Trial 00 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 192
activation: relu
learning_rate: 1e-05
Score: 0.9680122137069702

Trial 02 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 192
activation: relu
learning_rate: 0.001
Score: 0.9664889574050903

Trial 06 summary
Hyperparameters:
units_value1: 768
units_value2: 384
units_value3: 96
activation: relu
learning_rate: 1e-